# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) Student ID, Name and last name <br>
2) Student ID, Name and last name

In [10]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [9]:
model = LpProblem(name="my_model", sense=LpMaximize)
var_names = [str(_ + 1).upper() for _ in range(0, 3)]
x = LpVariable.dicts("x", var_names, 0,)

const_names = ["LE" + str(_ + 1) for _ in range(0, 4)] 
sense = [-1 for _ in range(0, 4)]
coefs = [[1,1,0], [0,2,1], [3,0,2], [1,1,1]]
rhs = [10, 12, 15, 20] 
for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(0, 3)])
    model += LpConstraint(e=expr, sense=s, name=cn, rhs=r)

obj_coefs = [4,2,3]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(0, 3)])
model

my_model:
MAXIMIZE
4*x_1 + 2*x_2 + 3*x_3 + 0
SUBJECT TO
LE1: x_1 + x_2 <= 10

LE2: 2 x_2 + x_3 <= 12

LE3: 3 x_1 + 2 x_3 <= 15

LE4: x_1 + x_2 + x_3 <= 20

VARIABLES
x_1 Continuous
x_2 Continuous
x_3 Continuous

In [13]:
status = model.solve()
for n in var_names:
    print(x[n].value())
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")


4.4285714
5.5714286
0.85714286
status: 1, Optimal
objective: 31.428571379999998


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [15]:
c = np.array([4, 2, 3])
b = np.array([[10],
              [12],
              [15],
              [20]])
A = np.array([[1, 1, 0],
              [0, 2, 1],
              [3, 0, 2],
              [1, 1, 1]])
# 4 equalities, 7 variables = 3 degrees of freedom

variables = [0, 1, 2, 3, 4, 5, 6]
row_zero = np.array([4, 2, 3, 0, 0, 0, 0])



<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [16]:
def get_table(basic_solution):
    M = np.concatenate((A, np.identity(len(basic_solution))), 1)

    try:
        one_solution = basic_solution  # all_basic_solutions[0]
        B = M[:, one_solution]
        Cb = row_zero[np.array(one_solution)]
        # print(one_solution)
        # print()
        # print(B)
        invB = np.linalg.inv(B)
        x_b = np.matmul(invB, b)
        row0_slacks_coefs = np.matmul(Cb, invB)
        row0_deciscion_coefs = np.matmul(np.matmul(Cb, invB), A) - c
        row1_deciscison = np.matmul(invB, A)
        row1_slacks = invB
        Z = np.matmul(Cb, x_b)[0]
        connected = np.concatenate((row1_deciscison, row1_slacks), 1)

        columns = ["Final", "Z"] + ["x_" + str(x + 1) for x in variables]
        # first_column = ["Z"] + ["x_" + str(x+1) for x in basic_solution]
        # rest_of_columns = [list(range(0, 5)) for x in range(0, 7)]
        df = pd.DataFrame(columns=columns)
        df.loc[0] = ["Z", 1] + list(row0_deciscion_coefs) + list(row0_slacks_coefs)
        # df= df.append(["Z", 1] + list(row0_deciscion_coefs) + list(row0_slacks_coefs))
        counter = 1
        for var_ in basic_solution:
            df.loc[counter] = ["x_" + str(var_ + 1), 0] + list(connected[counter - 1])
            counter += 1
        df["RS"] = [Z] + list(x[0] for x in np.matmul(invB, b))
        print(df.round(8))

        if min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)) >= -0.0000000001:
            print("Is optimal")
        else:
            # print(min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)))
            print("Is not optimal")

        if np.min(np.matmul(invB, b)) >= -0.0000000001:
            print("Is feasible")
        else:
            # print(min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)))
            print("Is not feasible")

    except LinAlgError:
        return None


In [17]:
all_basic_solutions = list(it.combinations(variables, 4))  # 3
get_table(all_basic_solutions[0])

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0  0.0  0.0  0.0 -1.0  0.0  4.0  68.0
1   x_1  0  1.0  0.0  0.0  0.0 -2.0 -1.0  4.0  41.0
2   x_2  0  0.0  1.0  0.0  0.0 -1.0 -1.0  3.0  33.0
3   x_3  0  0.0  0.0  1.0  0.0  3.0  2.0 -6.0 -54.0
4   x_4  0  0.0  0.0  0.0  1.0  3.0  2.0 -7.0 -64.0
Is not optimal
Is not feasible


In [19]:
for sol_ in all_basic_solutions:
    get_table(sol_)
    print()

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0  0.0  0.0  0.0 -1.0  0.0  4.0  68.0
1   x_1  0  1.0  0.0  0.0  0.0 -2.0 -1.0  4.0  41.0
2   x_2  0  0.0  1.0  0.0  0.0 -1.0 -1.0  3.0  33.0
3   x_3  0  0.0  0.0  1.0  0.0  3.0  2.0 -6.0 -54.0
4   x_4  0  0.0  0.0  0.0  1.0  3.0  2.0 -7.0 -64.0
Is not optimal
Is not feasible

  Final  Z  x_1  x_2  x_3       x_4  x_5       x_6       x_7         RS
0     Z  1  0.0  0.0  0.0  0.333333  0.0  0.666667  1.666667  46.666667
1   x_1  0  1.0  0.0  0.0  0.666667  0.0  0.333333 -0.666667  -1.666667
2   x_2  0  0.0  1.0  0.0  0.333333  0.0 -0.333333  0.666667  11.666667
3   x_3  0  0.0  0.0  1.0 -1.000000  0.0  0.000000  1.000000  10.000000
4   x_5  0  0.0  0.0  0.0  0.333333  1.0  0.666667 -2.333333 -21.333333
Is optimal
Is not feasible

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0  0.0  0.0  0.0 -1.0  0.0  4.0  68.0
1   x_1  0  1.0  0.0  0.0  0.5 -0.5  0.0  0.5   9.0
2   x_2  0  0.0  1.0  0.0  0.5  0.5  0.0

## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [11]:
model = LpProblem(name="my_fucking_problem", sense=LpMinimize)
var_names = [str(_ + 1).upper() for _ in range(0, 4)]
x = LpVariable.dicts("y", var_names, 0)

const_names = ["GE" + str(_ + 1) for _ in range(0, 3)]  # +["EQ1", "EQ2"]
sense = [1 for _ in range(0, 3)]  # + [0, 0]  # GE, LE, EQ
coefs = [[1, 0, 3, 1], [1, 2, 0, 1], [0, 1, 2, 1]]  # [6, -6, -1, -1],
# [-5, 3, 1, -5]]  # Matrix coefs
rhs = [4, 2, 3]

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(0, 4)])
    print(LpConstraint(e=expr, sense=s, name=cn, rhs=r))
    model += LpConstraint(e=expr, sense=s, name=cn, rhs=r)

obj_coefs = [10, 12, 15, 20]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(0, 4)])
print(model)
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names:
    print(x[n].value())

y_1 + 3*y_3 + y_4 >= 4
y_1 + 2*y_2 + y_4 >= 2
y_2 + 2*y_3 + y_4 >= 3
my_fucking_problem:
MINIMIZE
10*y_1 + 12*y_2 + 15*y_3 + 20*y_4 + 0
SUBJECT TO
GE1: y_1 + 3 y_3 + y_4 >= 4

GE2: y_1 + 2 y_2 + y_4 >= 2

GE3: y_2 + 2 y_3 + y_4 >= 3

VARIABLES
y_1 Continuous
y_2 Continuous
y_3 Continuous
y_4 Continuous

status: 1, Optimal
objective: 31.42857072
0.57142857
0.71428571
1.1428571
0.0


In [13]:
c = np.array([4, 2, 3])
b = np.array([[10],
              [12],
              [15],
              [20]])
A = np.array([[1, 1, 0],
              [0, 2, 1],
              [3, 0, 2],
              [1, 1, 1]])
# 4 equalities, 7 variables = 3 degrees of freedom

variables = [0, 1, 2, 3, 4, 5, 6]
row_zero = np.array([4, 2, 3, 0, 0, 0, 0])

# c = [4, 2]
# b = [[1],
#      [1],
#      [3]]
# A = [[0.5, -1],
#      [-2, 1],
#      [1, 1]]
# variables = [0, 1, 2, 3, 4]
# row_zero = np.array([4, 2, 0, 0, 0])


# print(np.identity(4))
tabelka = pd.DataFrame(
    columns=["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE",
             "D_STATE", "D_F", "D_O",
             "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])
# print(M)

new_row = {}
all_basic_solutions = list(it.combinations(variables, 4))  # 3
Zs = []
for basic_solution in all_basic_solutions:
    # print(basic_solution)
    M = np.concatenate((A, np.identity(len(basic_solution))), 1)
    try:
        one_solution = basic_solution  # all_basic_solutions[0]
        B = M[:, one_solution]
        Cb = row_zero[np.array(one_solution)]
        # print(one_solution)
        # print()
        # print(B)
        invB = np.linalg.inv(B)
        x_b = np.matmul(invB, b)
        row0_slacks_coefs = np.matmul(Cb, invB)
        row0_deciscion_coefs = np.matmul(np.matmul(Cb, invB), A) - c
        row1_deciscison = np.matmul(invB, A)
        row1_slacks = invB
        Z = np.matmul(Cb, x_b)[0]
        new_row["Z"] = Z
        connected = np.concatenate((row1_deciscison, row1_slacks), 1)

        Zs.append(Z)

        columns = ["Final", "Z"] + ["x_" + str(x + 1) for x in variables]
        # first_column = ["Z"] + ["x_" + str(x+1) for x in basic_solution]
        # rest_of_columns = [list(range(0, 5)) for x in range(0, 7)]
        df = pd.DataFrame(columns=columns)
        df.loc[0] = ["Z", 1] + list(row0_deciscion_coefs) + list(row0_slacks_coefs)
        # df= df.append(["Z", 1] + list(row0_deciscion_coefs) + list(row0_slacks_coefs))
        counter = 1
        for var_ in basic_solution:
            df.loc[counter] = ["x_" + str(var_ + 1), 0] + list(connected[counter - 1])
            counter += 1
        df["RS"] = [Z] + list(x[0] for x in np.matmul(invB, b))
        print(df.round(8))

        if min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)) >= -0.0000000001:
            new_row["P_O"] = True
            new_row["D_O"] = False
            print("Is optimal")
        else:
            # print(min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)))
            new_row["P_O"] = False
            new_row["D_O"] = True
            print("Is not optimal")

        if np.min(np.matmul(invB, b)) >= -0.0000000001:
            new_row["P_F"] = True
            new_row["D_F"] = False
            print("Is feasible")
        else:
            # print(min(list(row0_deciscion_coefs) + list(row0_slacks_coefs)))
            new_row["P_F"] = False
            new_row["D_F"] = True
            print("Is not feasible")

        if new_row["P_F"] and new_row["P_O"]:
            new_row["D_F"] = True
            new_row["D_O"] = True
            new_row["D_STATE"] = "Optimal"
            new_row["P_STATE"] = "Optimal"

        if new_row["P_O"] and not new_row["P_F"]:
            new_row["P_STATE"] = "Super"
            new_row["D_STATE"] = "Sub"

        if not new_row["P_O"] and new_row["P_F"]:
            new_row["P_STATE"] = "Sub"
            new_row["D_STATE"] = "Super"

        primal = {"x" + str(x + 1): y for x, y in zip(basic_solution, list(x[0] for x in np.matmul(invB, b)))}
        for slack_ in [x + 1 for x in variables if x not in basic_solution]:
            primal["x" + str(slack_)] = 0
        # print(basic_solution)
        primal = dict(sorted(primal.items()))
        print(primal)
        for key, value in primal.items():
            if int(key[1]) >= 4:
                new_row["slack" + str(int(key[1]) - 3)] = value
            else:
                new_row[key] = value

        dual = {key for key, value in primal.items() if value != 0}

        dual = {"y_" + str(6 - int(key[1])): 0 for key in dual}
        dual = dict(sorted(dual.items()))
        print(dual)

        counter_ = 1
        for x in np.matmul(Cb, invB):
            new_row["y" + str(counter_)] = x
            counter_ += 1
        counter_ = 1
        for x in Cb @ invB @ A - c:
            new_row["sur" + str(counter_)] = x
            counter_ += 1
        print(np.matmul(Cb, invB))
        print(Cb @ invB @ A - c)
        # new_row["D_STATE"] = "XDD"
        # new_row["P_STATE"] = "XDD2"
        print(new_row)
        print()
        # break
        tabelka = tabelka.append(new_row, ignore_index=True)
    except LinAlgError:
        continue
    # break
print(Zs)

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0  0.0  0.0  0.0 -1.0  0.0  4.0  68.0
1   x_1  0  1.0  0.0  0.0  0.0 -2.0 -1.0  4.0  41.0
2   x_2  0  0.0  1.0  0.0  0.0 -1.0 -1.0  3.0  33.0
3   x_3  0  0.0  0.0  1.0  0.0  3.0  2.0 -6.0 -54.0
4   x_4  0  0.0  0.0  0.0  1.0  3.0  2.0 -7.0 -64.0
Is not optimal
Is not feasible
{'x1': 41.00000000000003, 'x2': 33.00000000000003, 'x3': -54.00000000000006, 'x4': -64.00000000000006, 'x5': 0, 'x6': 0, 'x7': 0}
{'y_2': 0, 'y_3': 0, 'y_4': 0, 'y_5': 0}
[ 0.0000000e+00 -1.0000000e+00 -8.8817842e-16  4.0000000e+00]
[-8.8817842e-16  0.0000000e+00 -8.8817842e-16]
{'Z': 68.0, 'P_O': False, 'D_O': True, 'P_F': False, 'D_F': True, 'x1': 41.00000000000003, 'x2': 33.00000000000003, 'x3': -54.00000000000006, 'slack1': -64.00000000000006, 'slack2': 0, 'slack3': 0, 'slack4': 0, 'y1': 0.0, 'y2': -1.0000000000000009, 'y3': -8.881784197001252e-16, 'y4': 4.000000000000002, 'sur1': -8.881784197001252e-16, 'sur2': 0.0, 'sur3': -8.881784197001252e-

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0 -2.5  0.0 -0.5  0.0  1.5  0.0  17.5
1   x_1  0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  10.0
2   x_3  0  0.0 -1.5  1.0 -1.5  0.0  0.5  0.0  -7.5
3   x_5  0  0.0  3.5  0.0  1.5  1.0 -0.5  0.0  19.5
4   x_7  0  0.0  1.5  0.0  0.5  0.0 -0.5  1.0  17.5
Is not optimal
Is not feasible
{'x1': 10.0, 'x2': 0, 'x3': -7.5, 'x4': 0, 'x5': 19.5, 'x6': 0, 'x7': 17.5}
{'y_-1': 0, 'y_1': 0, 'y_3': 0, 'y_5': 0}
[-0.5  0.   1.5  0. ]
[ 0.  -2.5  0. ]
{'Z': 17.5, 'P_O': False, 'D_O': True, 'P_F': False, 'D_F': True, 'x1': 10.0, 'x2': 0, 'x3': -7.5, 'slack1': 0, 'slack2': 19.5, 'slack3': 0, 'slack4': 17.5, 'y1': -0.5, 'y2': 0.0, 'y3': 1.5, 'y4': 0.0, 'sur1': 0.0, 'sur2': -2.5, 'sur3': 0.0, 'P_STATE': 'Super', 'D_STATE': 'Sub'}

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1  0.0  8.0  0.0  4.0  3.0  0.0  0.0  76.0
1   x_1  0  1.0  1.0  0.0  1.0  0.0  0.0  0.0  10.0
2   x_3  0  0.0  2.0  1.0  0.0  1.0  0.0  0.0  12.0
3   x_6  0 

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1 -2.0  0.0 -1.0  0.0  0.0  0.0  2.0  40.0
1   x_2  0  1.0  1.0  1.0  0.0  0.0  0.0  1.0  20.0
2   x_4  0  0.0  0.0 -1.0  1.0  0.0  0.0 -1.0 -10.0
3   x_5  0 -2.0  0.0 -1.0  0.0  1.0  0.0 -2.0 -28.0
4   x_6  0  3.0  0.0  2.0  0.0  0.0  1.0  0.0  15.0
Is not optimal
Is not feasible
{'x1': 0, 'x2': 20.0, 'x3': 0, 'x4': -10.0, 'x5': -28.0, 'x6': 15.0, 'x7': 0}
{'y_0': 0, 'y_1': 0, 'y_2': 0, 'y_4': 0}
[0. 0. 0. 2.]
[-2.  0. -1.]
{'Z': 40.0, 'P_O': False, 'D_O': True, 'P_F': False, 'D_F': True, 'x1': 0, 'x2': 20.0, 'x3': 0, 'slack1': -10.0, 'slack2': -28.0, 'slack3': 15.0, 'slack4': 0, 'y1': 0.0, 'y2': 0.0, 'y3': 0.0, 'y4': 2.0, 'sur1': -2.0, 'sur2': 0.0, 'sur3': -1.0, 'P_STATE': 'Super', 'D_STATE': 'Sub'}

  Final  Z  x_1  x_2  x_3  x_4  x_5  x_6  x_7    RS
0     Z  1 -4.0  0.0 -2.0  0.0  1.0  0.0  0.0  12.0
1   x_2  0  0.0  1.0  0.5  0.0  0.5  0.0  0.0   6.0
2   x_4  0  1.0  0.0 -0.5  1.0 -0.5  0.0  0.0   4.0
3   x_6  0  3.0  0.

In [6]:
import numpy as np
import itertools as it
import pandas as pd

pd.set_option('display.chop_threshold', 0.00001)
pd.set_option("display.max_rows", None, "display.max_columns", None)

# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from numpy.linalg import LinAlgError

2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [20]:
### TODO 

### DISPLAY STORED PAIRS OF SOLUTIONS
#df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O",
#                                                   "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])

# display(df.style.set_properties(**{
#        'width': '230px',
#        'max-width': '230px',
#    }))
tabelka.round(3)

,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,D_O,y1,y2,y3,y4,sur1,sur2,sur3
0,68.000,41.000,33.000,-54.000,-64.00,0,0,0,False,False,NaN,NaN,True,True,0.000,-1.000,0.000,4.000,0.0,0.000,0.000
1,46.667,-1.667,11.667,10.000,0.00,-21.333333,0,0,False,True,Super,Sub,True,False,0.333,0.000,0.667,1.667,0.0,0.000,0.000
2,68.000,9.000,1.000,10.000,0.00,0,-32.0,0,False,False,Super,Sub,True,True,0.000,-1.000,0.000,4.000,0.0,0.000,0.000
3,31.429,4.429,5.571,0.857,0.00,0,0,9.142857,True,True,Optimal,Optimal,True,True,0.571,0.714,1.143,0.000,0.0,0.000,0.000
4,50.000,5.000,15.000,0.000,-10.00,-18.0,0,0,False,True,Super,Sub,True,False,0.000,0.000,0.667,2.000,0.0,0.000,0.333
5,68.000,14.000,6.000,0.000,-10.00,0,-27.0,0,False,False,Super,Sub,True,True,0.000,-1.000,0.000,4.000,0.0,0.000,0.000
6,32.000,5.000,6.000,0.000,-1.00,0,0,9.0,False,True,Super,Sub,True,False,0.000,1.000,1.333,0.000,0.0,0.000,0.667
7,30.000,5.000,5.000,0.000,0.00,2.0,0,10.0,True,False,Sub,Super,False,True,2.000,0.000,0.667,0.000,0.0,0.000,-1.667
8,28.000,4.000,6.000,0.000,0.00,0,3.0,10.0,True,False,Sub,Super,False,True,4.000,-1.000,0.000,0.000,0.0,0.000,-4.000
9,35.000,-25.000,0.000,45.000,35.00,-33.0,0,0,False,False,Sub,Super,True,True,0.000,0.000,1.000,1.000,0.0,-1.000,0.000
